In [1]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score, cross_validate
import multiprocessing

from sklearn.svm import SVC, SVR
from sklearn.manifold import TSNE
from sklearn.linear_model import SGDRegressor, SGDClassifier
from sklearn.datasets import load_boston, load_breast_cancer,load_iris
from sklearn.datasets import load_wine
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn import metrics

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 나이브 베이스 분류기(Naive Bayes Classification)

* 베이즈 정리를 적용한 확률적 분류 알고리즘
* 모든 특성들이 독립임을 가정 (naive 가정)
* 입력 특성에 따라 3개의 분류기 존재
  * 가우시안 나이브 베이즈 분류기
  * 베르누이 나이브 베이즈 분류기
  * 다항 나이브 베이즈 분류기

## 나이브 베이즈 분류기의 확률 모델

* 나이브 베이즈는 조건부 확률 모델
* *N*개의 특성을 나타내는 벡터 **x**를 입력 받아 k개의 가능한 확률적 결과를 출력

\begin{equation}
p(C_k | x_1,...,x_n)
\end{equation}

* 위의 식에 베이즈 정리를 적용하면 다음과 같음

\begin{equation}
p(C_k | \textbf{x}) = \frac{p(C_k)p(\textbf{x}|C_k)}{p(\textbf{x})}
\end{equation}

* 위의 식에서 분자만이 출력 값에 영향을 받기 때문에 분모 부분을 상수로 취급할 수 있음

\begin{equation}
\begin{split}
p(C_k | \textbf{x}) & \propto p(C_k)p(\textbf{x}|C_k) \\
& \propto p(C_k, x_1, ..., x_n)
\end{split}
\end{equation}

* 위의 식을 연쇄 법칙을 사용해 다음과 같이 쓸 수 있음

\begin{equation}
\begin{split}
p(C_k, x_1, ..., x_n) & = p(C_k)p(x_1, ..., x_n | C_k) \\
& = p(C_k)p(x_1 | C_k)p(x_2, ..., x_n | C_k, x_1) \\
& = p(C_k)p(x_1 | C_k)p(x_2 | C_k, x_1)p(x_3, ..., x_n | C_k, x_1, x_2) \\
& = p(C_k)p(x_1 | C_k)p(x_2 | C_k, x_1)...p(x_n | C_k, x_1, x_2, ..., x_{n-1})
\end{split}
\end{equation}

* 나이브 베이즈 분류기는 모든 특성이 독립이라고 가정하기 때문에 위의 식을 다음과 같이 쓸 수 있음

\begin{equation}
\begin{split}
p(C_k, x_1, ..., x_n) & \propto p(C_k)p(x_1|C_k)p(x_2|C_k)...p(x_n|C_k) \\
& \propto p(C_k) \prod_{i=1}^{n} p(x_i|C_k)
\end{split}
\end{equation}

* 위의 식을 통해 나온 값들 중 가장 큰 값을 갖는 클래스가 예측 결과

\begin{equation}
\hat{y} = \underset{k}{\arg\max} \; p(C_k) \prod_{i=1}^{n} p(x_i|C_k)
\end{equation}

- 정수형 또는 실수형: GaussianNB (1, 3, 2), (0.1, 0.9, 1.5)
- 정수형: MultinomialNB (1, 3, 2)
- 바이너리형: BernoulliNB (0,1,0)
- alpha : smoothing

In [2]:
weather=['Sunny','Sunny','Overcast','Rainy','Rainy','Rainy','Overcast','Sunny','Sunny',
'Rainy','Sunny','Overcast','Overcast','Rainy']
temp=['Hot','Hot','Hot','Mild','Cool','Cool','Cool','Mild','Cool','Mild','Mild','Mild','Hot','Mild']
play=['No','No','Yes','Yes','Yes','No','Yes','No','Yes','Yes','Yes','Yes','Yes','No']

In [6]:
df = pd.DataFrame( [weather,temp])
df = df.T
df.columns = ['날씨', '온도']
df['외출여부'] = play
df

,날씨,온도,외출여부
0,Sunny,Hot,No
1,Sunny,Hot,No
2,Overcast,Hot,Yes
3,Rainy,Mild,Yes
4,Rainy,Cool,Yes
5,Rainy,Cool,No
6,Overcast,Cool,Yes
7,Sunny,Mild,No
8,Sunny,Cool,Yes
9,Rainy,Mild,Yes


In [7]:
x_data = df.iloc[:,:-1]
y_data = df.iloc[:,-1]

In [9]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import make_column_transformer

In [10]:
ct = make_column_transformer( ( OneHotEncoder(), ['날씨','온도'] ) )

In [11]:
result = ct.fit_transform( df )
result

array([[0., 0., 1., 0., 1., 0.],
       [0., 0., 1., 0., 1., 0.],
       [1., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1.],
       [0., 1., 0., 1., 0., 0.],
       [0., 1., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 1.],
       [0., 0., 1., 1., 0., 0.],
       [0., 1., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 1.]])

In [13]:
ct.get_feature_names()

['onehotencoder__x0_Overcast',
 'onehotencoder__x0_Rainy',
 'onehotencoder__x0_Sunny',
 'onehotencoder__x1_Cool',
 'onehotencoder__x1_Hot',
 'onehotencoder__x1_Mild']

In [14]:
model_pipe = make_pipeline(  ct, BernoulliNB() )
model_pipe.fit( x_data, y_data )

Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['날씨', '온도'])],
                                   verbose=False)),
                ('bernoullinb',
                 BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None,
                             fit_prior=True))],


In [16]:
testdf = pd.DataFrame( [['Sunny','Hot']], columns=['날씨', '온도']  )
testdf

,날씨,온도
0,Sunny,Hot


In [19]:
model_pipe.predict( testdf )

array(['No'], dtype='<U3')

## wine 데이터셋 나이브베이즈를 이용하여 test[0] 데이터에 대한 분류를
# 출력하고, score 를 구하시요.

In [22]:
wine = load_wine()
df = pd.DataFrame(wine.data, columns=wine.feature_names)
x_train, x_test, y_train, y_test = train_test_split(wine.data, wine.target, stratify=wine.target, test_size=0.2)
model_pipe = make_pipeline( StandardScaler(), GaussianNB())
param_grid={"gaussiannb__var_smoothing":np.linspace(0.1, 1, 10)}
gridS = GridSearchCV(model_pipe, param_grid, scoring='f1_macro')
# model_pipe.fit(x_train, y_train)
gridS.fit(x_train, y_train)
print( gridS.best_params_ )
print( gridS.best_estimator_.predict([x_test[0]]) )
print( gridS.best_score_ )

{'gaussiannb__var_smoothing': 0.2}
[1]
0.964875590510804


In [23]:
wine = load_wine()

x_data = wine['data']
y_data = wine['target']
X_train, X_test, y_train, y_test = train_test_split( x_data, 
                                                    y_data, 
                                                    test_size=0.2, 
                                                    stratify=y_data)
y_data #다항분류
# 가우시안 
model_wine = make_pipeline( StandardScaler(), GaussianNB() )
model_wine.fit( X_train, y_train )
# 예측
model_wine.predict( [X_test[0]] )
# 스코어
model_wine.score( X_test, y_test )

0.9444444444444444